### Generate private key

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa


key = rsa.generate_private_key(
     public_exponent=65537,
     key_size=2048,
     backend=default_backend()
 )

# write key to disk. passphrase assumed to be admin

with open("/home/ftp/crypto_gen_key.pem", "wb") as f:   f.write(key.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,
                              encryption_algorithm=serialization.BestAvailableEncryption(b"admin"),))
    
    
#########################
# create decrypted private key (save with no passphrase)
#########################


from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

key = rsa.generate_private_key(
     public_exponent=65537,
     key_size=2048,
     backend=default_backend()
 )
print key.key_size # key size
# Write our key to disk for safe keeping
with open("/home/ftp/crypto_gen_key_plain.pem", "wb") as f:
    f.write(key.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,
                              encryption_algorithm=serialization.NoEncryption(),))
    

#########################
#decrypt an existing private key and save to disk
#########################

with open("/home/ftp/certificates/clientkey4096.pem", "r") as f:      
    pemdata=f.read()
    
clientcertkey = load_pem_private_key(data=pemdata,password='admin',backend=default_backend())  

with open("/home/ftp/clientkey4096.pem", "wb") as f:
    f.write(clientcertkey.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,encryption_algorithm=serialization.NoEncryption(),))
    
    
    
#############################
# load key from disk as crypto obj
#############################

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import load_pem_private_key

with open("/home/ftp/ocsp.key", "r") as f:
    pemdata=f.read()
key = load_pem_private_key(data=pemdata,password='admin',backend=default_backend()) # passphrase admin
print isinstance(key, rsa.RSAPrivateKey) 
print key.key_size
print key.public_key() 


### Generate certificate and sign it

In [ ]:

from cryptography.hazmat.backends import default_backend
import datetime
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

### Load private key, passphrase is assumed admin
with open("/home/ftp/crypto_gen_key.pem", "r") as f:
    pemdata=f.read()
key = load_pem_private_key(data=pemdata,password='admin',backend=default_backend())


# certificate details
subject = issuer = x509.Name([x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),x509.NameAttribute(NameOID.COMMON_NAME, u"mysite.com"),])

# add extensions to cert
cert = x509.CertificateBuilder().subject_name(subject).issuer_name(issuer).public_key(key.public_key()).serial_number(
    x509.random_serial_number()).not_valid_before(datetime.datetime.utcnow()).not_valid_after(
    datetime.datetime.utcnow()+\
    datetime.timedelta(days=10)).add_extension(
    x509.SubjectAlternativeName([x509.DNSName(u"localhost")]),critical=False,).sign(key, hashes.SHA1(), default_backend())

# Write cert to disk
with open("/home/ftp/crypto_certificate.pem", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))

### Customized extensions

In [ ]:

custom_oid = x509.ObjectIdentifier('1.3.6.1.4.1.34380.1.1.13') # custom extension

csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
            x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
            x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),
            x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
            x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
            x509.NameAttribute(NameOID.COMMON_NAME, u"mysite.com"),
        ])).add_extension(
                x509.UnrecognizedExtension(custom_oid, b'foobar'),
        critical=False,
        # Sign the CSR with our private key.
        ).sign(key, hashes.SHA256(), default_backend())

### generate a Root CA cert and intermediate cert

In [ ]:
# rootca
from cryptography.hazmat.backends import default_backend
import datetime
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

#create private key
key = rsa.generate_private_key(
     public_exponent=65537,
     key_size=4096,
     backend=default_backend()
 )

with open("/home/ftp/certificates/bloom/rootcakey4096.pem", "wb") as f:
    f.write(key.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,
                              encryption_algorithm=serialization.BestAvailableEncryption(b"admin"),))


subject = issuer = x509.Name([x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),x509.NameAttribute(NameOID.COMMON_NAME, u"Systems Security RootCA"),])

cert = x509.CertificateBuilder().subject_name(subject).issuer_name(issuer).public_key(key.public_key()).serial_number(
    x509.random_serial_number()).not_valid_before(datetime.datetime.utcnow()).not_valid_after(
    datetime.datetime.utcnow()+\
    datetime.timedelta(days=700)).add_extension(
    x509.SubjectAlternativeName([x509.DNSName(u"RootCA")]),critical=False,).add_extension(x509.BasicConstraints(ca=True, path_length=None),critical=False).sign(key, hashes.SHA1(), default_backend())


with open("/home/ftp/certificates/bloom/rootcacert4096.pem", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))

################################################
# intermidiate cert
################################################

#create private key
key = rsa.generate_private_key(
     public_exponent=65537,
     key_size=4096,
     backend=default_backend()
 )


with open("/home/ftp/certificates/bloom/subrootcakey4096.pem", "wb") as f:
    f.write(key.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,
                              encryption_algorithm=serialization.BestAvailableEncryption(b"admin"),))


subject = x509.Name([x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),x509.NameAttribute(NameOID.COMMON_NAME, u"Data subRootCA"),])

issuer= x509.Name([x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),x509.NameAttribute(NameOID.COMMON_NAME, u"Systems Security RootCA"),])

with open("/home/ftp/certificates/bloom/rootcakey4096.pem", "r") as f:
          
    pemdata=f.read()
    
rootkey = load_pem_private_key(data=pemdata,password='admin',backend=default_backend())

cert = x509.CertificateBuilder().subject_name(subject).issuer_name(issuer).public_key(key.public_key()).serial_number(
    x509.random_serial_number()).not_valid_before(datetime.datetime.utcnow()).not_valid_after(
    datetime.datetime.utcnow()+\
    datetime.timedelta(days=650)).add_extension(
    x509.SubjectAlternativeName([x509.DNSName(u"subRootCA")]),critical=False,).add_extension(x509.BasicConstraints(ca=True, path_length=0),critical=False).sign(rootkey, hashes.SHA256(), default_backend())


with open("/home/ftp/certificates/bloom/subrootcacert4096.pem", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))
    


### signing client and OSCP certs with our generate root or intermediate root certs

In [ ]:
keyclient = rsa.generate_private_key(
     public_exponent=65537,
     key_size=4096,
     backend=default_backend()
 )

with open("/home/ftp/certificates/bloom/clientkey4096.pem", "wb") as f:
    f.write(keyclient.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,
                              encryption_algorithm=serialization.BestAvailableEncryption(b"admin"),))
    
with open("/home/ftp/certificates/bloom/clientkey4096clear.pem", "wb") as f:
    f.write(keyclient.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,
                              encryption_algorithm=serialization.NoEncryption(),))
    

keyocsp = rsa.generate_private_key(
     public_exponent=65537,
     key_size=4096,
     backend=default_backend()
 )
with open("/home/ftp/certificates/bloom/ocspkey4096.pem", "wb") as f:
    f.write(keyocsp.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,
                              encryption_algorithm=serialization.BestAvailableEncryption(b"admin"),))

with open("/home/ftp/certificates/bloom/ocspkey4096clear.pem", "wb") as f:
    f.write(keyocsp.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.TraditionalOpenSSL,
                              encryption_algorithm=serialization.NoEncryption(),))    
    
issuer = x509.Name([x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),x509.NameAttribute(NameOID.COMMON_NAME, u"Data subRootCA"),])

with open("/home/ftp/certificates/bloom/subrootcakey4096.pem", "r") as f:
          
    pemdata=f.read()
    
subrootkey = load_pem_private_key(data=pemdata,password='admin',backend=default_backend())    

#cert client
clientsubject=x509.Name([x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),x509.NameAttribute(NameOID.COMMON_NAME, u"338:DLWS"),])

clientcert = x509.CertificateBuilder().subject_name(clientsubject).issuer_name(issuer).public_key(keyclient.public_key()).serial_number(
    x509.random_serial_number()).not_valid_before(datetime.datetime.utcnow()).not_valid_after(
    datetime.datetime.utcnow()+\
    datetime.timedelta(days=650)).add_extension(
    x509.SubjectAlternativeName([x509.DNSName(u"clientcert")]),critical=False,).sign(
    subrootkey,hashes.SHA256(),default_backend())


with open("/home/ftp/certificates/bloom/clientcert4096.pem", "wb") as f:
    f.write(clientcert.public_bytes(serialization.Encoding.PEM))
    
ocspsubject=x509.Name([x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),x509.NameAttribute(NameOID.COMMON_NAME, u"ocsp4096"),])

ocspcert = x509.CertificateBuilder().subject_name(ocspsubject).issuer_name(issuer).public_key(keyocsp.public_key()).serial_number(
    x509.random_serial_number()).not_valid_before(datetime.datetime.utcnow()).not_valid_after(
    datetime.datetime.utcnow()+\
    datetime.timedelta(days=650)).add_extension(
    x509.SubjectAlternativeName([x509.DNSName(u"ocsp")]),critical=False,).sign(
    subrootkey,hashes.SHA256(),default_backend())


with open("/home/ftp/certificates/bloom/ocspcert4096.pem", "wb") as f:
    f.write(ocspcert.public_bytes(serialization.Encoding.PEM))


### signing key with pkcs15

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

message = b"A message I want to sign"
with open("/home/ftp/ocsp.key", "r") as f:         
    pemdata=f.read()
print pemdata
 
private_key = load_pem_private_key(data=pemdata,password=None,backend=default_backend())  
signature = private_key.sign(message,padding.PKCS1v15(),hashes.SHA256())

public_key = private_key.public_key()
print public_key.verify(signature,message,padding.PKCS1v15(),hashes.SHA256())

### Load and explore cert attributes

In [ ]:


from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.backends.openssl.backend import backend
import datetime
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.x509.extensions import Extension
from cryptography.x509 import UnrecognizedExtension
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
import binascii

with open("/home/ftp/certificates/firstcert.bin", "r") as f:
    pemdata=f.read()      

###########
# load DER format cert
###########
cert = x509.load_der_x509_certificate(pemdata, default_backend())

################
# explore attributes and extensions
###############
cert_prop={}
cert_prop['subject_name']=cert.subject 
cert_prop['serial_number']=cert.serial_number
cert_prop['not_valid_before']=cert.not_valid_before
cert_prop['not_valid_after']=cert.not_valid_after
cert_prop['extensions']=cert.extensions

print type(cert.extensions), type(cert.not_valid_after), type(cert.serial_number), type(cert.subject)

for e in cert.extensions:
    print e, '\n'
    
for a in cert.subject:
    print  a.value, a.oid.dotted_string
    


### Load cert from pcaps

In [ ]:
# loading der cert from capture
# export from cap as bin or der first
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.backends.openssl.backend import backend
import datetime
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography import x509
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.x509.oid import NameOID
from cryptography.x509.extensions import Extension
from cryptography.x509 import UnrecognizedExtension
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
import binascii

####### from pcap- export cert as packet bytes

hexx="30:82:03:58:30:82:02:40:a0:03:02:01:02:02:04:60:93:b5:86:30:0d:06:09:2a:86:48:86:f7:0d:01:01:0b:05:00:30:0c:31:0a:30:08:06:03:55:04:03:0c:01:79:30:1e:17:0d:32:31:30:35:30:36:30:39:33:36:30:31:5a:17:0d:32:32:30:35:30:36:30:39:33:36:30:31:5a:30:0c:31:0a:30:08:06:03:55:04:03:0c:01:79:30:82:01:22:30:0d:06:09:2a:86:48:86:f7:0d:01:01:01:05:00:03:82:01:0f:00:30:82:01:0a:02:82:01:01:00:a8:2d:f7:8c:1b:5e:e6:c8:2f:3d:f6:c8:c0:f6:8b:d7:49:06:69:8e:5d:52:37:9c:f5:c0:b1:aa:27:9d:2a:da:27:34:d6:a3:68:d5:d9:5c:39:95:f3:75:6f:30:58:e3:0d:16:9b:e0:d7:16:67:ab:6d:53:82:dd:d8:9c:bc:0f:8f:2d:d0:5a:06:b7:82:37:f3:32:e2:38:04:4c:77:cf:11:d9:00:b6:41:91:cb:01:6e:c3:7f:b4:7a:89:48:35:8e:68:99:31:0f:6d:80:e3:ea:49:a4:94:44:d6:d1:b8:0f:00:1c:3b:94:54:b0:90:ca:fb:94:6c:b4:f9:71:22:4c:85:63:9b:4b:7c:a0:eb:30:f0:ec:1e:1d:4b:0c:a8:cd:a9:77:c3:f6:12:91:56:bc:cd:2e:34:92:5e:2e:cd:78:ac:e0:7e:b6:2e:12:af:98:d9:9f:12:ca:17:0a:c0:c0:87:26:59:38:51:47:24:66:9c:40:19:77:c8:a3:a2:85:4a:08:60:5d:ae:3d:13:6b:47:cd:26:38:9a:e8:0e:cc:d5:99:fc:be:35:af:74:68:bb:77:f8:36:7d:d1:4b:0d:b0:c2:45:d7:02:4b:ef:35:0d:4e:84:36:01:d9:ea:be:06:5f:fa:b6:d6:00:77:e4:b5:e2:5a:a2:7c:be:ab:02:03:01:00:01:a3:81:c1:30:81:be:30:0f:06:03:55:1d:13:01:01:ff:04:05:30:03:01:01:ff:30:11:06:09:60:86:48:01:86:f8:42:01:01:04:04:03:02:02:44:30:33:06:09:60:86:48:01:86:f8:42:01:0d:04:26:16:24:41:6c:74:65:6f:6e:2f:52:61:64:77:61:72:65:20:47:65:6e:65:72:61:74:65:64:20:43:65:72:74:69:66:69:63:61:74:65:30:1d:06:03:55:1d:0e:04:16:04:14:99:b4:ff:cf:a9:1b:57:59:f1:19:ae:44:7f:58:0d:2e:d1:e3:11:af:30:37:06:03:55:1d:23:04:30:30:2e:80:14:99:b4:ff:cf:a9:1b:57:59:f1:19:ae:44:7f:58:0d:2e:d1:e3:11:af:a1:10:a4:0e:30:0c:31:0a:30:08:06:03:55:04:03:0c:01:79:82:04:60:93:b5:86:30:0b:06:03:55:1d:0f:04:04:03:02:02:ec:30:0d:06:09:2a:86:48:86:f7:0d:01:01:0b:05:00:03:82:01:01:00:11:a2:03:42:30:61:59:9d:92:4e:f2:3e:dd:58:a5:4a:cf:0e:df:43:b2:b0:16:65:19:02:b8:5f:e4:b2:62:49:9c:a3:4b:74:aa:ec:26:d5:ae:d7:c4:e5:9c:39:b2:99:3f:fe:30:c5:5c:0d:4b:ae:7e:69:93:99:40:9e:bd:ff:a4:6a:65:4e:b9:7b:4d:8e:2e:34:9b:b2:70:da:c9:13:51:f5:c0:b4:0b:11:ee:e0:8d:65:43:51:1e:06:89:99:b2:b5:8a:d4:b4:c8:b4:23:44:0f:4c:0a:82:59:76:5b:14:fa:25:53:fa:bd:30:f2:9e:b4:8a:b4:6c:ff:4d:ce:ea:b4:de:b3:8c:0b:96:08:97:34:62:90:c2:66:29:3f:f0:d5:46:32:e4:21:ca:cd:49:d4:74:18:4a:8f:4d:d1:92:0d:29:dc:3b:22:df:c6:76:e1:38:27:68:7c:42:de:5c:0d:87:d0:8a:7d:ee:c4:ec:05:f3:f9:01:2b:21:69:b7:e8:88:e1:a9:64:c8:82:50:33:0d:be:ef:e9:e5:90:25:ba:17:c4:c5:6e:fa:53:1a:03:bb:60:d2:21:72:b9:4d:bd:a2:13:45:38:78:dd:ee:94:d4:d5:a9:dc:60:3d:1b:77:1a:55:dd:db:9a:3d:5e:f5:e9:f4:2b:5e:2c:51"

hexx=hexx.replace(":","")
cert = x509.load_der_x509_certificate(bu, default_backend())
print 'serial number:',cert.serial_number,'\n', type(cert.serial_number)
print cert.subject , type(cert.subject)
print cert.issuer, type(cert.issuer)
print cert.version
print cert.signature_algorithm_oid
print cert.public_key().key_size
